In [22]:
import pyomo.environ as pe
from pyomo.opt import SolverFactory
from pyomo.environ import DataPortal
import pandas as pd
import time

model=pe.AbstractModel()

model.I =pe.Set()
model.J =pe.Set()
model.P =pe.Set()
model.K =pe.Param(model.I, within=pe.Any)
def ik_init(model):
    return ((i,k) for i in model.I for k in model.K[i])
model.IK = pe.Set(dimen=2, initialize=ik_init)

model.C_ikj = pe.Param(model.IK, model.J, within=pe.Reals)
model.F_ik = pe.Param(model.IK, within=pe.NonNegativeIntegers)
model.C_ip = pe.Param(model.I, model.P, within=pe.Reals)
model.D_jp = pe.Param(model.J, model.P, within=pe.NonNegativeIntegers)
model.X_ik_max = pe.Param(model.IK, within=pe.NonNegativeIntegers)
model.X_ik_min = pe.Param(model.IK, within=pe.NonNegativeIntegers)
model.X_p = pe.Param(model.P, within=pe.NonNegativeIntegers)
model.Q_i = pe.Param(model.I, within=pe.NonNegativeIntegers)
model.S_ijp = pe.Param(model.I, model.J, model.P, within=pe.NonNegativeIntegers)

 
model.x_ikjp = pe.Var(model.IK, model.J, model.P, domain=pe.NonNegativeIntegers, bounds=(0, 10000000))
model.z_ik = pe.Var(model.IK, domain=pe.Binary)

data = DataPortal()
data.load(filename='C:/Users/DELL PC/Desktop/Polyhouse/PolyhouseBig.dat', model=model)


def obj_expr(model):
    return sum(model.C_ikj[i, k, j] * sum(model.x_ikjp[i,k,j,p] for p in model.P) for (i,k) in model.IK for j in model.J ) + sum(model.F_ik[i,k]*model.z_ik[i,k] for (i,k) in model.IK ) + sum(model.C_ip[i,p]*model.x_ikjp[i,k,j,p] for (i,k) in model.IK for j in model.J for p in model.P)

model.OBJ = pe.Objective(rule=obj_expr)

def c1_rule(model, j, p):
    return sum(model.S_ijp[i,j,p]*sum(model.x_ikjp[i,k,j,p] for k in model.K[i]) for i in model.I) == model.D_jp[j,p]
model.c1 = pe.Constraint(model.J, model.P, rule=c1_rule)

def c2_rule(model, i, k, j, p):
    return model.x_ikjp[i,k,j,p] >= 0
model.c2 = pe.Constraint(model.IK, model.J, model.P, rule=c2_rule)

def c3_rule(model, i, k, j, p):
    return model.x_ikjp[i,k,j,p] <= model.D_jp[j,p]
model.c3 = pe.Constraint(model.IK, model.J, model.P, rule=c3_rule)

def c4_rule(model, i, k):
    return sum(model.x_ikjp[i,k,j,p] for j in model.J for p in model.P) >= model.X_ik_min[i,k]*model.z_ik[i,k]
model.c4 = pe.Constraint(model.IK, rule=c4_rule)

def c5_rule(model, i, k):
    return sum(model.x_ikjp[i,k,j,p] for j in model.J for p in model.P) <= model.X_ik_max[i,k]*model.z_ik[i,k]
model.c5 = pe.Constraint(model.IK, rule=c5_rule)

def c6_rule(model, i):
    return sum((sum(model.x_ikjp[i,k,j,p] for k in model.K[i] for j in model.J)/model.X_p[p]) for p in model.P) <= model.Q_i[i]
model.c6 = pe.Constraint(model.I, rule=c6_rule)


instance1 = model.create_instance(data)
print("Minimum required area of farming land:")
print(sum(sum((instance1.D_jp[j, p]/instance1.X_p[p]) for p in instance1.P) for j in instance1.J))


Minimum required area of farming land:
303.33000000000027


In [23]:
solver = SolverFactory('gurobi')
start_time = time.time()
results = solver.solve(instance1)
end_time = time.time()
print("Runtime: {:.2f} seconds".format(end_time - start_time))
print('Objective value:', instance1.OBJ())

Runtime: 5623.73 seconds
Objective value: 284280.44628023007


In [30]:
import pyomo.environ as pe
from pyomo.opt import SolverFactory
from pyomo.environ import DataPortal
import itertools
def solve_tsp(Cities,Mode):

    model = pe.AbstractModel()
    model.cities = pe.Set(initialize=Cities)
    model.sub_cities = pe.Set(initialize=Cities[1:])
    model.CITIES = pe.Set()
    model.transport_mode = pe.Set(initialize=[1,2,3,4])
    model.DISTANCE = pe.Param(model.CITIES, model.CITIES, within=pe.Reals)  
    model.L = pe.Param(model.transport_mode, within=pe.Reals, initialize={1:(1000), 2:(2000), 3:(6000), 4:(6000)})
    
    data = DataPortal()
    data.load(filename='C:/Users/DELL PC/Desktop/Polyhouse/DistanceBig.dat', model=model)
    
    data_dict = {}
    for i in Cities:
        for j in Cities:
            data_dict[(i,j)] = data['DISTANCE'][i,j]
    model.d_ij = pe.Param(model.cities, model.cities, within=pe.Reals, initialize=data_dict)
    
    Z_dict = {}
    for i in Cities:
        for j in Cities:
            if i == Cities[0] and j == Cities[0]:
                Z_dict[j] = 0
            if i == Cities[0] and j != Cities[0] and sum(instance1.x_ikjp[i,Mode,j,p].value for p in instance1.model().P) < 1:
                Z_dict[j] = 0
            if i == Cities[0] and j != Cities[0] and sum(instance1.x_ikjp[i,Mode,j,p].value for p in instance1.model().P) > 1:
                Z_dict[j] = sum(instance1.x_ikjp[i,Mode,j,p].value for p in instance1.model().P)
    
    model.Z =  pe.Param(model.cities, within=pe.Reals, initialize=Z_dict)
 
    model.y = pe.Var(model.cities, model.cities, within=pe.Binary)
    model.w = pe.Var(model.cities, model.cities, within=pe.NonNegativeIntegers)
    model.u = pe.Var(model.cities, within=pe.NonNegativeReals)
    
    
    def obj_expression(model):
        return sum(model.d_ij[i,j]*model.w[i,j] for i in model.cities for j in model.sub_cities) + model.L[Mode]*sum(model.d_ij[i,j]*model.y[i,j] for i in model.cities for j in model.sub_cities) + model.L[Mode]*sum(model.d_ij[i,Cities[0]]*model.y[i,Cities[0]] for i in model.sub_cities)
    model.obj = pe.Objective(expr=obj_expression, sense=pe.minimize)

    
    def con1_rule(model, i):
        return sum(model.y[i,j] for j in model.cities if j != i) == 1
    model.con1 = pe.Constraint(model.cities, rule=con1_rule)

    def con2_rule(model, j):
        return sum(model.y[i,j] for i in model.cities if i != j) == 1
    model.con2 = pe.Constraint(model.cities, rule=con2_rule)

    def no_self_loops(model, i):
        return model.y[i,i] == 0
    model.no_self_loops = pe.Constraint(model.cities, rule=no_self_loops)
    
    
    # Define the MTZ subtour elimination constraint
    def mtz_subtour_elimination_rule(model, i, j):
        if i != j and i!=Cities[0]:
            return model.u[j] - model.u[i] + model.y[i,j]*len(Cities) <= len(Cities) - 1 
        else:
            return pe.Constraint.Skip
    model.mtz_subtour_elimination = pe.Constraint(model.cities, model.cities, rule=mtz_subtour_elimination_rule)
    
    
    def con6_rule(model,i,j):
        if i != j:
            return model.w[i,j] >= sum(model.w[k,i] for k in model.cities if k != i and k!= j) - model.Z[i] - sum(model.Z[k] for k in model.cities)*(1-model.y[i,j])
        else:
            return pe.Constraint.Skip
    model.con6 = pe.Constraint(model.sub_cities, model.sub_cities, rule=con6_rule)

    def con7_rule(model,j):
        return model.w[Cities[0],j] == model.y[Cities[0],j]*sum(model.Z[i] for i in model.cities)
    model.con7 = pe.Constraint(model.sub_cities, rule=con7_rule)
    
    def con8_rule(model,i,j):
        if i != j:
            return model.w[i,j] >= model.Z[j]*model.y[i,j]
        else:
            return pe.Constraint.Skip
    model.con8 = pe.Constraint(model.sub_cities, model.sub_cities, rule=con8_rule)

    def con9_rule(model,i,j):
        if i != j:
            return model.w[i,j] <= model.y[i,j]*sum(model.Z[k] for k in model.cities)
        else:
            return pe.Constraint.Skip
    model.con9 = pe.Constraint(model.sub_cities, model.sub_cities, rule=con9_rule)

    
    instance2 = model.create_instance()
    solver = SolverFactory('gurobi')
    
    start_time1 = time.time()
    
    results = solver.solve(instance2)
    end_time1 = time.time()

    for i in instance2.model().cities:
        for j in instance2.model().cities:
            if instance2.y[i,j].value > 0:
                print(f"y[{i},{j}]: {instance2.y[i,j].value}")
    

In [43]:
# Create an empty dictionary to store the sets of unique j values
j_dict = {}

# Loop over the i, k, and j sets
for (i,k) in instance1.model().IK:
    for j in instance1.model().J:
        for p in instance1.model().P:
            # Check if x[i,k,j,p] is greater than zero in the solution
            if instance1.x_ikjp[i,k,j,p].value > 1:
                # Add j to the set for the corresponding i and k
                key = (i,k)
                if key not in j_dict:
                    j_dict[key] = set()
                # If i is not in the set, add it to index 0
                if i not in j_dict[key]:
                    j_dict[key].add(i)
                # Add j to the set
                j_dict[key].add(j)
    # Convert the set of unique j values to a list and store it in the dictionary, if key exists
    key = (i,k)
    if key in j_dict:
        j_list = list(j_dict[key])
        # If i is in the list, move it to index 0
        if i in j_list:
            j_list.remove(i)
            j_list.insert(0, i)
        j_dict[key] = j_list


In [44]:
for key in j_dict:
    print(key, ':', j_dict[key])
    print('Total Cities in the route:', len(j_dict[key]))
    if len(j_dict[key]) > 2:
        solve_tsp(j_dict[key],key[1])
    print('')

('HaldwaniP', 3) : ['HaldwaniP', 'HaldwaniDC', 'BhojipuraDC', 'NainitalDC']
Total Cities in the route: 4
y[HaldwaniP,HaldwaniDC]: 1.0
y[HaldwaniDC,NainitalDC]: 1.0
y[BhojipuraDC,HaldwaniP]: 1.0
y[NainitalDC,BhojipuraDC]: 1.0

('RanipokhariP', 3) : ['RanipokhariP', 'MussorieDC', 'RishikeshDC']
Total Cities in the route: 3
y[RanipokhariP,RishikeshDC]: 1.0
y[MussorieDC,RanipokhariP]: 1.0
y[RishikeshDC,MussorieDC]: 1.0

('SitarganjP', 3) : ['SitarganjP', 'BhojipuraDC', 'SitarganjDC']
Total Cities in the route: 3
y[SitarganjP,SitarganjDC]: 1.0
y[BhojipuraDC,SitarganjP]: 1.0
y[SitarganjDC,BhojipuraDC]: 1.0

('DoiwalaP', 3) : ['DoiwalaP', 'DehradunDC']
Total Cities in the route: 2

('SelakuiP', 2) : ['SelakuiP', 'VikasnagarDC']
Total Cities in the route: 2

('SelakuiP', 3) : ['SelakuiP', 'PaontasahibDC', 'VikasnagarDC']
Total Cities in the route: 3
y[SelakuiP,VikasnagarDC]: 1.0
y[PaontasahibDC,SelakuiP]: 1.0
y[VikasnagarDC,PaontasahibDC]: 1.0

('RoorkeeP', 3) : ['RoorkeeP', 'NahanDC', 'Haridw

y[SandilaP,LucknowDC]: 1.0
y[LucknowDC,KanpurDC]: 1.0
y[ShahjahanpurDC,LakhimpurDC]: 1.0
y[LakhimpurDC,SandilaP]: 1.0
y[KannaujDC,ShahjahanpurDC]: 1.0
y[KanpurDC,KannaujDC]: 1.0

('HandiaP', 4) : ['HandiaP', 'JaunpurDC', 'RamnagarDC', 'PrayagrajDC', 'MirzapurDC']
Total Cities in the route: 5
y[HandiaP,PrayagrajDC]: 1.0
y[JaunpurDC,RamnagarDC]: 1.0
y[RamnagarDC,HandiaP]: 1.0
y[PrayagrajDC,MirzapurDC]: 1.0
y[MirzapurDC,JaunpurDC]: 1.0

('ShivpurP', 4) : ['ShivpurP', 'VaranasiDC']
Total Cities in the route: 2

('ChunnarP', 4) : ['ChunnarP', 'VaranasiDC']
Total Cities in the route: 2



In [33]:
print('Objective value:', instance1.OBJ())

Objective value: 284280.44628023007


In [45]:
for (i,k) in instance1.model().IK:
    for j in instance1.model().J:
        if sum(instance1.x_ikjp[i,k,j,p].value for p in instance1.model().P) > 1:
            print(f"x[{i},{k},{j}]: {sum(instance1.x_ikjp[i,k,j,p].value for p in instance1.model().P)}")

x[HaldwaniP,3,HaldwaniDC]: 2175.0
x[HaldwaniP,3,NainitalDC]: 1850.0
x[HaldwaniP,3,BhojipuraDC]: 1974.9999999998734
x[RanipokhariP,3,RishikeshDC]: 1850.0
x[RanipokhariP,3,MussorieDC]: 4150.0
x[SitarganjP,3,SitarganjDC]: 2550.0
x[SitarganjP,3,BhojipuraDC]: 3450.0
x[DoiwalaP,3,DehradunDC]: 6000.0
x[SelakuiP,2,VikasnagarDC]: 2000.0
x[SelakuiP,3,VikasnagarDC]: 2400.0
x[SelakuiP,3,PaontasahibDC]: 3600.0
x[RoorkeeP,3,RoorkeeDC]: 1850.0
x[RoorkeeP,3,HaridwarDC]: 3600.0
x[RoorkeeP,3,NahanDC]: 550.0
x[SrinagarP,2,SrinagarDC]: 2000.0
x[SrinagarP,3,SrinagarDC]: 6000.0
x[KotdwarP,1,KotdwarDC]: 1000.0
x[HaridwarP,3,HaridwarDC]: 6000.0
x[PithoragarhP,2,PithoragarhDC]: 1960.0
x[PithoragarhP,2,LohaghatDC]: 40.0
x[KhatimaP,4,KhatimaDC]: 1850.0
x[KhatimaP,4,PilibhitDC]: 980.0
x[KhatimaP,4,LohaghatDC]: 110.0
x[KhatimaP,4,BhojipuraDC]: 4570.0
x[KhatimaP,4,ShahjahanpurDC]: 490.0
x[HaldwaniP,4,BhojipuraDC]: 8000.0
x[SitarganjP,4,BhojipuraDC]: 8000.0
x[KichhaP,4,KichhaDC]: 620.0
x[KichhaP,4,BareillyDC]: 6974.

In [35]:
df = pd.DataFrame(columns=['Polyhouse', 'Transport Mode', 'Distribution Center', 'Product', 'x'])
for (i, k, j, p), var in instance1.x_ikjp.items():
    x_value = pe.value(var)
    if x_value > 0:
        df1 = pd.DataFrame({'Polyhouse': [i], 'Transport Mode': [k], 'Distribution Center': [j], 'Product': [p], 'x': [x_value]})
        df = pd.concat([df, df1], ignore_index=True)

print(df)

pd.set_option('display.max_rows', None)

        Polyhouse Transport Mode Distribution Center      Product  \
0       HaldwaniP              3          HaldwaniDC       Tomato   
1       HaldwaniP              3          HaldwaniDC     Capsicum   
2       HaldwaniP              3          HaldwaniDC   BellPepper   
3       HaldwaniP              3          HaldwaniDC     Cucumber   
4       HaldwaniP              3          HaldwaniDC  GreenChilli   
5       HaldwaniP              3          RanikhetDC       Tomato   
6       HaldwaniP              3          NainitalDC       Tomato   
7       HaldwaniP              3          NainitalDC     Capsicum   
8       HaldwaniP              3          NainitalDC   BellPepper   
9       HaldwaniP              3          NainitalDC     Cucumber   
10      HaldwaniP              3          NainitalDC  GreenChilli   
11      HaldwaniP              3         BhojipuraDC     Capsicum   
12      HaldwaniP              3         BhojipuraDC     Cucumber   
13      HaldwaniP              3  

In [36]:
df2 = df[df['x'] > 0].groupby('Polyhouse')['x'].sum().to_frame().T
df3 = df[df['x'] > 0].groupby('Distribution Center')['x'].sum().to_frame().T
df4 = df[df['x'] > 0].groupby('Product')['x'].sum().to_frame().T
df5 = df[df['x'] > 0].groupby('Transport Mode')['x'].sum().to_frame().T



In [37]:
df.to_excel("C:\\Users\\DELL PC\\Desktop\\ResultM.xlsx")
df=pd.read_excel("C:\\Users\\DELL PC\\Desktop\\ResultM.xlsx")

In [38]:

df2.to_excel("C:\\Users\\DELL PC\\Desktop\\Outputi.xlsx")
df2=pd.read_excel("C:\\Users\\DELL PC\\Desktop\\Outputi.xlsx")
df4.to_excel("C:\\Users\\DELL PC\\Desktop\\Outputp.xlsx")
df4=pd.read_excel("C:\\Users\\DELL PC\\Desktop\\Outputp.xlsx")
df5.to_excel("C:\\Users\\DELL PC\\Desktop\\Outputk.xlsx")
df5=pd.read_excel("C:\\Users\\DELL PC\\Desktop\\Outputk.xlsx")

In [39]:
df_pivoted = df.set_index(['Polyhouse', 'Transport Mode', 'Distribution Center', 'Product'])['x'].unstack('Transport Mode')
print(df_pivoted)


Transport Mode                                     1       2             3  \
Polyhouse    Distribution Center Product                                     
AmethiP      AyodhyaDC           GreenChilli     NaN     NaN           NaN   
BadlapurP    JaunpurDC           BellPepper      NaN   350.0           NaN   
                                 Cucumber        NaN   150.0           NaN   
                                 Tomato          NaN  1500.0           NaN   
BaghpatP     DelhiDC             BellPepper      NaN     NaN  1.450000e+03   
                                 Capsicum        NaN     NaN           NaN   
                                 GreenChilli     NaN     NaN  1.000000e+03   
                                 Tomato          NaN  2000.0  3.550000e+03   
BikapurP     AyodhyaDC           BellPepper      NaN     NaN  1.000000e+02   
                                 Capsicum        NaN     NaN  5.000000e+01   
                                 Cucumber        NaN     NaN  3.

In [40]:
df_pivoted.to_excel("C:\\Users\\DELL PC\\Desktop\\Pivoted.xlsx")
df_pivoted=pd.read_excel("C:\\Users\\DELL PC\\Desktop\\Pivoted.xlsx")

In [41]:
idx = df_pivoted.index[df_pivoted.iloc[:, 1].ne(df_pivoted.iloc[:, 1].shift())].tolist()

# create a new dataframe with the empty rows inserted
new_df = pd.DataFrame(columns=df_pivoted.columns)
start_idx = 0
for i in idx:
    new_df = new_df.append(df_pivoted.iloc[start_idx:i, :])
    new_df = new_df.append(pd.Series(), ignore_index=True)
    start_idx = i
new_df = new_df.append(df_pivoted.iloc[start_idx:, :])

# write the new dataframe to a new excel file
new_df.to_excel("C:\\Users\\DELL PC\\Desktop\\NewPivoted.xlsx", index=False)

C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df_pivoted.iloc[start_idx:i, :])
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df_pivoted.iloc[start_idx:i, :])
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df_pivoted.iloc[start_idx:i, :])
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df_pivoted.iloc[start_idx:i, :])
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df_pivoted.iloc[start_idx:i, :])
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

...

C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df_pivoted.iloc[start_idx:i, :])
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df_pivoted.iloc[start_idx:i, :])
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype expl

C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df_pivoted.iloc[start_idx:i, :])
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df_pivoted.iloc[start_idx:i, :])
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df_pivoted.iloc[start_idx:i, :])
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(pd.Series(), ignore_index=True)
C:\Users\DELL PC\AppData\Local\Temp\ipykernel_6600\1418346411.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste